# Time Series Modeling

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from random import gauss as gs
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.pylab import rcParams
%matplotlib inline

## White Noise

The idea behind white noise is that it is truly random.

We don't want white noise to describe our model per se, but we *do* want it to describe our model *error*.

Can you explain these truisms?

In [ ]:
# Let's make some white noise!

rands = []
for _ in range(1000):
    rands.append(gs(0, 1))
series = pd.Series(rands)

In [ ]:
X = np.linspace(-10, 10, 1000)
plt.figure(figsize=(10, 7))
plt.plot(X, series);

What happens if we do a seasonal decomposition on this?

In [ ]:
series.index[:5]

In [ ]:
time_index = pd.to_datetime(series.index, unit='D')

In [ ]:
time_index[:5]

In [ ]:
series.index = time_index
decomp_white = sm.tsa.seasonal_decompose(series)

In [ ]:
plt.figure(figsize=(20, 8))
plt.subplot(411)
plt.plot(series.index, decomp_white.observed, label='Original')
plt.legend()
plt.subplot(412)
plt.plot(series.index, decomp_white.trend, label='Trend')
plt.legend()
plt.subplot(413)
plt.plot(series.index, decomp_white.seasonal, label='Seasonal')
plt.legend()
plt.subplot(414)
plt.plot(series.index, decomp_white.resid, label='Residual')
plt.legend()
plt.subplots_adjust(hspace=0.5);

## Football Data

In [ ]:
fb = pd.read_csv('data/google-trends_football_us.csv').iloc[1:, :]
fb.columns = ['counts']

In [ ]:
fb['counts'] = fb['counts'].replace('<1', '0')
fb['counts'] = fb['counts'].astype(int)

In [ ]:
fb.head()

### Series as Both Predictor and Target?

Often, the phenomenon we want to capture with a time series is a dataset being correlated with *itself*.

Well, of course every dataset is perfectly correlated with itself. But what we're after now is the idea that a series is correlated with *earlier versions* of itself.

Consider the problem of trying to predict tomorrow's closing price for some stock on the market. One may consider lots of features, like what sort of company it is to which the stock belongs or whether that company has been in the news recently.

But it is very often the case that one of the most helpful predictors of tomorrow's price is *today's* price. And so we want to build a model where one of our predictors is an earlier version of our target.

One tool we can use is **`df.rolling()`**, which creates a Rolling object that we can use to calculate statistics dynamically.

In [ ]:
type(fb.rolling(window=1))

In [ ]:
fb.rolling(window=3).mean().head()

In [ ]:
fb['roll_avg'] = fb.rolling(window=2).mean()

fb.corr()

In [ ]:
fb['roll_avg'].head()

In [ ]:
plt.scatter(fb.index[:30], fb['counts'][:30])
plt.scatter(fb.index[1:31], fb['roll_avg'][1:31]);

In [ ]:
lr = LinearRegression()

lr.fit(fb[['roll_avg']][1:], fb['counts'][1:])

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(fb.index[1:25], fb['counts'][1:25], label='Data')
plt.plot(fb.index[1:25], lr.predict(fb[['roll_avg']][1:25]),
         label='Predicted')
plt.legend();

### Autocorrelation and Partial Autocorrelation Functions

Pandas and statsmodels offer autocorrelation (ACF) and partial autocorrelation (PACF) plotting tools. The idea here is to look at the correlation of a series with itself for some particular interval or *lag*. The key difference between the full and the partial autocorrelation functions is that the partial autocorrelation function ignores intervening intervals. For more, see [this post](https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/).

#### Autocorrelation

The basic idea of autocorrelation is simple: See how a series correlates with a "lagged" version of itself. If my sequence is $S_0 = (x_0, x_1, x_2, ... , x_n)$, then I can measure the Pearson correlation between the first $n-k + 1$ terms of $S_0$ and $S_{lag} = (x_k, x_{k+1}, x_{k+2}, ... , x_n)$.

In [ ]:
acf(fb['counts'], nlags=20, fft=False)

In [ ]:
# Plot using `.autocorr()`

X = np.arange(0, 100, 1)
Y = np.zeros(100)
for j in X[1:]:
    Y[j] = fb['counts'].autocorr(lag=j)
plt.figure(figsize=(20, 4))
plt.plot(X, Y)
plt.grid();

In [ ]:
# To construct the autocorrelation function, we take the covariance of our time series with a lagged version
# and then divide by the variance of the series.

X = np.arange(0, 100, 1)
Y = np.zeros(100)
for j in X[1:]:
    Y[j] += np.cov(fb['counts'][:-j], fb['counts'][j:])[0, 1] / np.var(fb['counts']) * ((180-j) / 180)
plt.figure(figsize=(20, 4))
plt.plot(X, Y)
plt.grid();

In [ ]:
plt.figure(figsize=(20, 4))
pd.plotting.autocorrelation_plot(fb['counts']);

The horizontal bands represent condfidence intervals, which are calculated by taking relevant z-scores of the standard normal distribution and dividing by the square root of the number of observations. For more, see [here](https://www.itl.nist.gov/div898/handbook/eda/section3/autocopl.htm).

Calculation of the 95%-confidence interval:

In [ ]:
stats.norm().ppf(0.975) / np.sqrt(fb['counts'].shape[0])

Calculation of the 99%-confidence interval:

In [ ]:
stats.norm().ppf(0.995) / np.sqrt(fb['counts'].shape[0])

We can also use the `plot_acf()` function from statsmodels:

In [ ]:
rcParams['figure.figsize'] = 20, 4

plot_acf(fb['counts'], lags=125, alpha=None);

#### Partial Autocorrelation

The idea behind partial Autocorrelation is to compare a series to a lagged version of itself while abstracting away from intermediate values. In effect, this amounts to exploring the correlations among *residuals*.

In [ ]:
pacf(fb['counts'], nlags=20)

##### Calculation #1

One way of computing the partial autocorrelation is by fitting regressions to residuals from a simple dummy model that always predicts the mean. The coefficient of the final term will be the partial autocorrelation for the corresponding number of lags.

In [ ]:
y_tilde = fb['counts'] - fb['counts'].mean()

In [ ]:
x_1 = (fb['counts'][:-1] - fb['counts'].mean()).values.reshape(-1, 1)
x_2 = (fb['counts'][:-2] - fb['counts'].mean()).values.reshape(-1, 1)

In [ ]:
lr = LinearRegression()

lr.fit(np.concatenate([x_1[1:], x_2], axis=1), y_tilde[2:]).coef_[-1]

In [ ]:
x_1 = (fb['counts'][:-1] - fb['counts'].mean()).values.reshape(-1, 1)
x_2 = (fb['counts'][:-2] - fb['counts'].mean()).values.reshape(-1, 1)
x_3 = (fb['counts'][:-3] - fb['counts'].mean()).values.reshape(-1, 1)

In [ ]:
lr2 = LinearRegression()

lr2.fit(np.concatenate([x_1[2:], x_2[1:], x_3], axis=1), y_tilde[3:]).coef_[-1]

##### Calculation #2

An alternative way of calculating these values is to solve the matrix equation:

$\begin{bmatrix}
\rho(0) & ... & \rho(k-1) \\
... & ... & ... \\
\rho(k-1) & ... & \rho(0)
\end{bmatrix}
\begin{bmatrix}
\phi_{k1} \\
... \\
\phi_{kk}
\end{bmatrix} = \begin{bmatrix}
\rho(1) \\
... \\
\rho(k)
\end{bmatrix}$,

where $\rho(k)$ is the autocorrelation for $k$ lags.

In [ ]:
row1 = acf(fb['counts'], nlags=1, fft=False)
row2 = acf(fb['counts'], nlags=1, fft=False)[::-1]

autos = np.vstack([row1, row2])
autos

In [ ]:
b = acf(fb['counts'], nlags=2, fft=False)[1:]

In [ ]:
np.linalg.solve(autos, b)[-1]

For more on this method, see [this post](https://stats.stackexchange.com/questions/129052/acf-and-pacf-formula).

In [ ]:
rcParams['figure.figsize'] = 20, 4

plot_pacf(fb['counts'], lags=20, alpha=0.05);

For more on ACF and PACF, see [these slides](http://people.cs.pitt.edu/~milos/courses/cs3750/lectures/class16.pdf) and [this post](https://towardsdatascience.com/significance-of-acf-and-pacf-plots-in-time-series-analysis-2fa11a5d10a8).

## ARMA Modeling

- 'AR' is for "**Auto-Regressive**": The prediction for today will be a function of the value for previous days.

    The number of lag periods we want to include will be a parameter in the statsmodels model object ("p").

    In particular, auto-regressive models look like this:

    $X_t = \beta_0 + \Sigma^p_{i=1}\beta_iX_{t-i} + \epsilon_t$, <br/>
    where $\epsilon_t$ should be more or less accurately modeled by white noise.

    We indicate how many terms our $AR$ model has by writing $AR(k)$ where $k$ is the number of terms.

    Looking at the PACF can help us decide on an appropriate $p$: We can look at where the correlation values cross the confidence thresholds. <br/><br/>

- 'MA' is for "**Moving Average**": The prediction for today will be a function of the rolling mean.

    The number of average terms we want to include will be a parameter in the statsmodels model object ("q").

    In  particular, moving-average models look like this:

    $X_t = \mu + \epsilon_t + \Sigma^q_{i=1}\beta_i\epsilon_{t-i}$, <br/>
    where again the $\epsilon$ should be modeled by white noise.

    We indicate how many terms our $MA$ model has by writing $MA(k)$ where $k$ is the number of terms.

    Looking at the ACF can help us decide on an appropriate $q$: We can look at where the correlation values cross the confidence thresholds.

For some technical details, see [this page](https://en.wikipedia.org/wiki/Autoregressive%E2%80%93moving-average_model#Choosing_p_and_q).

The $AR$ and $MA$ models are intimately related. In fact $AR(p)$ is equivalent to $MA(\infty)$ for any $p$. The reverse holds as well if $|\theta| < 1$ for all $\theta$ in $MA(q)$. For more on this, see [here](https://otexts.com/fpp2/MA.html).

Consider $AR(1)$:

$X_t = \beta_0 + \beta_1X_{t-1} + \epsilon_t$ <br/> 
$= \beta_0 + \beta_1(\beta_1X_{t-2} + \epsilon_{t-1})$ <br/>
$= \beta_0 + \beta_1^2X_{t-2} + \beta_1\epsilon_{t-1}$ <br/>
$= \beta_0 + \beta_1^3X_{t-3} + \beta_1^2\epsilon_{t-2} + \beta_1\epsilon_{t-1}$

In the limit of this expansion we obtain an expression for $MA(\infty)$.

### Stationarity and the Dickey-Fuller Test

ARMA models assume that the time series is *stationary*, which means that its statistical properties are not a (meaningful) function of time.

It may seem counterintuitive that, for modeling purposes, we want our time series not to be a function of time! But the basic idea is the familiar one that we want our datapoints to be mutually *independent*. For more on this topic, see [here](https://stats.stackexchange.com/questions/19715/why-does-a-time-series-have-to-be-stationary).

One way of testing for stationarity is to use the Dickey-Fuller Test. The statsmodels version returns the test statistic and a p-value, relative to the null hypothesis that the series in question is NOT stationary. For more, see [this Wikipedia page](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test).

In [ ]:
# Presumably, our football series is not stationary. Let's check.

adfuller(fb['counts'], autolag=None)

But let's check the stationarity of the *differences* of our data. We'll use **`.diff()`**:

In [ ]:
fb['counts'].head()

In [ ]:
fb['counts'].diff().head()

In [ ]:
adfuller(fb['counts'].diff()[1:], autolag=None)

### Building the Model

In [ ]:
p = 3
q = 1

# This model will have three auto-regressive terms and one moving-average term.

ar = ARMA(fb['counts'].diff().values[1:], (p, q)).fit()

In [ ]:
ar.summary()

In [ ]:
np.sqrt(mean_squared_error(fb['counts'].diff().values[1:], ar.predict()))

In [ ]:
r2_score(fb['counts'].diff().values[1:], ar.predict())

#### `np.cumsum()`
Let's use `np.cumsum()` to add up our predictions!

In [ ]:
fb['counts'].head()

In [ ]:
np.cumsum(fb['counts']).head()

In [ ]:
preds = ar.predict()
full = fb['counts'].values[0] +  np.cumsum(preds)

f, a = plt.subplots(figsize=(20, 4))
a.plot(fb.index[1:], fb['counts'][1:], 'r', label='Data')
a.plot(fb.index[1:-1], full[1:], 'k', label='Fit')
plt.legend();

In [ ]:
f, a = plt.subplots(figsize=(20, 4))
a.plot(fb.index[1:], fb['counts'].diff()[1:], label='Data')
a.plot(fb.index[1:-1], preds[1:], label='Fit')
plt.legend();

### Unemployment Data

In [ ]:
data = pd.read_csv('data/seasonally-adjusted-quarterly-us.csv')

In [ ]:
data.head()

In [ ]:
data.columns = ['year_q', 'unemp_rate']
data['unemp_rate'] = data['unemp_rate'].map(lambda x:\
                                            float(str(x).replace('%', '')))
data.dropna(inplace=True)

In [ ]:
data['date'] = pd.to_datetime(data['year_q']).dt.to_period('Q')
data.set_index('date', inplace=True, drop=True)

In [ ]:
data.head()

In [ ]:
p = 2
q = 0
ar2 = ARMA(data['unemp_rate'].diff()[1:].values, (p, q)).fit()

In [ ]:
preds2 = ar2.predict()
full2 = data['unemp_rate'].values[0] + np.cumsum(preds2)

f, a = plt.subplots(figsize=(20, 4))
a.plot(data.index.to_timestamp()[1:], data['unemp_rate'][1:],
       'r', label='Data')
a.plot(data.index.to_timestamp()[1:], full2, 'k', label='Fit')
plt.legend();

In [ ]:
data['unemp_rate'][-5:]

In [ ]:
# `.forecast()`: Let's check out the doc here.

ar2.forecast(steps=3)

In [ ]:
data['unemp_rate'].tail()

In [ ]:
data['unemp_rate'].head()

In [ ]:
data.head()

In [ ]:
data['unemp_rate'][-1]

In [ ]:
data['unemp_rate'][-1] + np.cumsum(ar2.forecast(steps=3)[0])

In [ ]:
f, a = plt.subplots(figsize=(20, 4))
a.plot(data.index.to_timestamp()[1:], data['unemp_rate'].diff()[1:],
       label='Data')
a.plot(data.index.to_timestamp()[1:], preds2, label='Fit')
plt.legend();

### ARMA $\rightarrow$ ARIMA $\rightarrow$ SARIMA $\rightarrow$ SARIMAX

This idea of using the *differences* of our data points to make our data stationary and so appropriate for ARMA modeling is encoded in **ARIMA** modeling. The 'I' is for "integrated". **SARIMA** modeling adds a **S**easonal component. And **SARIMAX** modeling adds e**X**ogenous (independent) variables.

Statsmodels supports these models as well:

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

#### Let's see if we can do a better job with our football data by including a seasonal component.

In [ ]:
season_model = SARIMAX(fb['counts'], order=(3, 1, 1), freq='MS').fit()

season_model.summary()

In [ ]:
season_model.predict()

In [ ]:
fb['counts']

In [ ]:
np.sqrt(mean_squared_error(fb['counts'], season_model.predict()))

Is this worse than before? Or not?

In [ ]:
r2_score(fb['counts'], season_model.predict())

In [ ]:
f, a = plt.subplots(figsize=(10, 5))

a.plot(fb.index, fb['counts'])
a.plot(fb.index, season_model.predict());